# Natural Language Processing Final Assignment

## Real - Fake News Classification: A Comparison of Natural Language Models for Classification

In [1]:
# Import Libraries

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import (
    classification_report,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)
from sklearn.pipeline import make_pipeline
from sklearn.base import clone
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

In [2]:
# Load Data

df_raw = pd.read_csv("data/news_dataset.csv")

In [3]:
df_raw.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [4]:
df = df_raw[['title', 'label']]

In [5]:
df = df.sample(frac=1).reset_index(drop=True)

In [6]:
df.head()

,title,label
0,House ethics panel investigating Democrat Corr...,1
1,Spain sacks Catalan government after independe...,1
2,California Senate passes gun control package,1
3,EU seeks to end Brexit transition no later tha...,1
4,Another Trump Official Decides It’s A REALLY ...,0


### Preprocessing Function

In [7]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/jakobhren/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jakobhren/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jakobhren/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
def preprocess_text(text):
    
    text = text.str.lower()
    
    # this remove links and twitter handels, and emails etc
    text = text.str.replace(r'http\S+|www\S+|https\S+', '', regex=True)
    text = text.str.replace(r'\S+@\S+', '', regex=True)
    text = text.str.replace(r'@\w+', '', regex=True)
    
    text = text.str.replace(f'[{string.punctuation}]', '', regex=True)
    
    text = text.str.replace(r'\d+', '', regex=True)
    
    text = text.str.replace(r'\s+', ' ', regex=True).str.strip()
    
    text_tokens = text.apply(nltk.word_tokenize)
    
    stop_words = set(stopwords.words('english'))
    text_tokens = text_tokens.apply(lambda tokens: [w for w in tokens if w not in stop_words])
    
    lemmatizer = WordNetLemmatizer()
    text_tokens = text_tokens.apply(lambda tokens: [lemmatizer.lemmatize(w) for w in tokens])
    
    processed_text = text_tokens.apply(lambda tokens: ' '.join(tokens))
    
    return processed_text


In [9]:
df["feature"] = preprocess_text(df["title"])

### Train - Test Split

In [10]:
X = df.feature
y = df.label

In [11]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, stratify=y, random_state=7)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, stratify=y_temp, random_state=7)



X_train = X_train.reset_index(drop=True)
X_val = X_val.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_val = y_val.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [12]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(27110,)
(5809,)
(5810,)


## Model Fitting

### Define functions for efficient model evaluation and fitting

In [13]:
def evaluate_models(y_true, y_pred_dict, label=None):
    """
    Evaluate and compare multiple models' predictions.
    Parameters:
    y_true : array-like
        True labels.
    y_pred_dict : dict
        Dictionary where keys are model names and values are predicted labels.
    label : str, optional
        Description of the evaluation set (e.g., "Validation Set").
    Returns:
    pd.DataFrame containing accuracy, precision, recall, and F1 for each model.
    """

    metrics = {
        'Model': [],
        'Accuracy': [],
        'Precision': [],
        'Recall': [],
        'F1-score': []
    }

    for model_name, y_pred in y_pred_dict.items():
        print(f"\n===== {model_name}: {label if label else ''} =====")
        print("Classification Report:")
        print(classification_report(y_true, y_pred))

        metrics['Model'].append(model_name)
        metrics['Accuracy'].append(accuracy_score(y_true, y_pred))

        unique_labels = np.unique(y_true)
        if len(unique_labels) == 2:
            metrics['Precision'].append(precision_score(y_true, y_pred, average='binary', pos_label=unique_labels[1]))
            metrics['Recall'].append(recall_score(y_true, y_pred, average='binary', pos_label=unique_labels[1]))
            metrics['F1-score'].append(f1_score(y_true, y_pred, average='binary', pos_label=unique_labels[1]))
        else:
            metrics['Precision'].append(precision_score(y_true, y_pred, average='weighted'))
            metrics['Recall'].append(recall_score(y_true, y_pred, average='weighted'))
            metrics['F1-score'].append(f1_score(y_true, y_pred, average='weighted'))

    return pd.DataFrame(metrics)


In [14]:
def fit_pipeline_predict_evaluate(
    model_class,
    model_name,
    X_train,
    y_train,
    X_val,
    y_val,
    vectorizer_type='tfidf',
    ngram_range=(1,1)
):
    if vectorizer_type == 'tfidf':
        vectorizer = TfidfVectorizer(stop_words='english', ngram_range=ngram_range)
    elif vectorizer_type == 'bow':
        vectorizer = CountVectorizer(stop_words='english', ngram_range=ngram_range)
    elif vectorizer_type is None:
        vectorizer = None
    else:
        raise ValueError("vectorizer_type must be either 'tfidf' or 'bow'")

    if vectorizer:
        pipeline = make_pipeline(vectorizer, model_class)
    else:
        pipeline = model_class

    print(f"Fitting {model_name}...")
    pipeline.fit(X_train, y_train)

    print(f"Predicting on training set...")
    y_pred_train = pipeline.predict(X_train)

    print(f"Predicting on validation set...")
    y_pred_val = pipeline.predict(X_val)

    print(f"Evaluating {model_name} on training set...")
    train_metrics = evaluate_models(y_train, {model_name: y_pred_train}, label="Train Set")

    print(f"Evaluating {model_name} on validation set...")
    val_metrics = evaluate_models(y_val, {model_name: y_pred_val}, label="Validation Set")

    train_metrics["Dataset"] = "Train"
    val_metrics["Dataset"] = "Validation"

    return pd.concat([train_metrics, val_metrics], ignore_index=True)

### Bag Of Words + Unigrams Vecotrizer

In [15]:
logistic_bow_uni_metrics = fit_pipeline_predict_evaluate(
    LogisticRegression(penalty='l2', C=0.1,max_iter=1000),
    "Logistic Regression (BoW)",
    X_train,
    y_train,
    X_val,
    y_val,
    vectorizer_type='bow',
    ngram_range=(1, 1)
)
print(logistic_bow_uni_metrics)

Fitting Logistic Regression (BoW)...
Predicting on training set...
Predicting on validation set...
Evaluating Logistic Regression (BoW) on training set...

===== Logistic Regression (BoW): Train Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.90      0.93     12532
           1       0.92      0.97      0.95     14578

    accuracy                           0.94     27110
   macro avg       0.94      0.94      0.94     27110
weighted avg       0.94      0.94      0.94     27110

Evaluating Logistic Regression (BoW) on validation set...

===== Logistic Regression (BoW): Validation Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.88      0.91      2685
           1       0.90      0.96      0.93      3124

    accuracy                           0.92      5809
   macro avg       0.93      0.92      0.92      5809
weighted avg       0.92      0.92    

In [17]:
bow_uni_nb_metrics = fit_pipeline_predict_evaluate(
    model_class=MultinomialNB(alpha=3),
    model_name="Multinomial Naive Bayes (BoW Unigrams)",
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    vectorizer_type='bow',
    ngram_range=(1, 1)
)

print(bow_uni_nb_metrics)

Fitting Multinomial Naive Bayes (BoW Unigrams)...
Predicting on training set...
Predicting on validation set...
Evaluating Multinomial Naive Bayes (BoW Unigrams) on training set...

===== Multinomial Naive Bayes (BoW Unigrams): Train Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.94      0.94     12532
           1       0.95      0.94      0.95     14578

    accuracy                           0.94     27110
   macro avg       0.94      0.94      0.94     27110
weighted avg       0.94      0.94      0.94     27110

Evaluating Multinomial Naive Bayes (BoW Unigrams) on validation set...

===== Multinomial Naive Bayes (BoW Unigrams): Validation Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.94      0.93      2685
           1       0.95      0.93      0.94      3124

    accuracy                           0.93      5809
   macro avg       0.93   

In [16]:
bow_uni_rf_metrics = fit_pipeline_predict_evaluate(
    model_class= RandomForestClassifier(
    n_estimators=100,
    max_depth=120,
    min_samples_leaf=5,
    max_features='sqrt',
    random_state=7),
    model_name="Random Forest (BoW Unigrams)",
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    vectorizer_type='bow',
    ngram_range=(1, 1)
)

print(bow_uni_rf_metrics)

Fitting Random Forest (BoW Unigrams)...
Predicting on training set...
Predicting on validation set...
Evaluating Random Forest (BoW Unigrams) on training set...

===== Random Forest (BoW Unigrams): Train Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.84      0.89     12532
           1       0.88      0.96      0.91     14578

    accuracy                           0.90     27110
   macro avg       0.91      0.90      0.90     27110
weighted avg       0.91      0.90      0.90     27110

Evaluating Random Forest (BoW Unigrams) on validation set...

===== Random Forest (BoW Unigrams): Validation Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.84      0.88      2685
           1       0.87      0.94      0.91      3124

    accuracy                           0.90      5809
   macro avg       0.90      0.89      0.89      5809
weighted avg       0.9

In [20]:
bow_uni_mlp_metrics = fit_pipeline_predict_evaluate(
    model_class=MLPClassifier(
    hidden_layer_sizes=(32,),
    activation='relu',
    solver='adam',
    alpha=0.5,
    learning_rate='adaptive',
    max_iter=100,
    early_stopping=True,
    n_iter_no_change=5,
    random_state=7
),
    model_name="MLP",
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    vectorizer_type='bow',
    ngram_range=(1, 1)
)
print(bow_uni_mlp_metrics)

Fitting MLP...
Predicting on training set...
Predicting on validation set...
Evaluating MLP on training set...

===== MLP: Train Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.94      0.95     12532
           1       0.95      0.98      0.96     14578

    accuracy                           0.96     27110
   macro avg       0.96      0.96      0.96     27110
weighted avg       0.96      0.96      0.96     27110

Evaluating MLP on validation set...

===== MLP: Validation Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.90      0.92      2685
           1       0.92      0.95      0.94      3124

    accuracy                           0.93      5809
   macro avg       0.93      0.93      0.93      5809
weighted avg       0.93      0.93      0.93      5809

  Model  Accuracy  Precision    Recall  F1-score     Dataset
0   MLP  0.958871   0.949456  0

### Bag of Words + Bigrams Vecotrizer:

In [21]:
logistic_bow_bigrams_metrics = fit_pipeline_predict_evaluate(
    LogisticRegression(penalty='l2', C=0.05,max_iter=1000),
    "Logistic Regression (BoW + Bigrams)",
    X_train,
    y_train,
    X_val,
    y_val,
    vectorizer_type='bow',
    ngram_range=(1, 2)
)
print(logistic_bow_bigrams_metrics)

Fitting Logistic Regression (BoW + Bigrams)...
Predicting on training set...
Predicting on validation set...
Evaluating Logistic Regression (BoW + Bigrams) on training set...

===== Logistic Regression (BoW + Bigrams): Train Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.90      0.94     12532
           1       0.92      0.98      0.95     14578

    accuracy                           0.94     27110
   macro avg       0.95      0.94      0.94     27110
weighted avg       0.94      0.94      0.94     27110

Evaluating Logistic Regression (BoW + Bigrams) on validation set...

===== Logistic Regression (BoW + Bigrams): Validation Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.86      0.91      2685
           1       0.89      0.97      0.93      3124

    accuracy                           0.92      5809
   macro avg       0.92      0.91      0.

In [22]:
bow_bi_nb_metrics = fit_pipeline_predict_evaluate(
    model_class=MultinomialNB(alpha=10),
    model_name="Multinomial Naive Bayes (BoW Bigrams)",
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    vectorizer_type='bow',
    ngram_range=(1, 2)
)

print(bow_bi_nb_metrics)

Fitting Multinomial Naive Bayes (BoW Bigrams)...
Predicting on training set...
Predicting on validation set...
Evaluating Multinomial Naive Bayes (BoW Bigrams) on training set...

===== Multinomial Naive Bayes (BoW Bigrams): Train Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.95      0.95     12532
           1       0.96      0.95      0.95     14578

    accuracy                           0.95     27110
   macro avg       0.95      0.95      0.95     27110
weighted avg       0.95      0.95      0.95     27110

Evaluating Multinomial Naive Bayes (BoW Bigrams) on validation set...

===== Multinomial Naive Bayes (BoW Bigrams): Validation Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.93      0.92      2685
           1       0.94      0.91      0.93      3124

    accuracy                           0.92      5809
   macro avg       0.92      0.

In [23]:
bow_bi_rf_metrics = fit_pipeline_predict_evaluate(
    model_class=RandomForestClassifier(
    n_estimators=100,
    max_depth=120,
    min_samples_leaf=5,
    max_features='sqrt',
    random_state=7),
    model_name="Random Forest (BoW Bigrams)",
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    vectorizer_type='bow',
    ngram_range=(1, 2)
)

print(bow_bi_rf_metrics)

Fitting Random Forest (BoW Bigrams)...
Predicting on training set...
Predicting on validation set...
Evaluating Random Forest (BoW Bigrams) on training set...

===== Random Forest (BoW Bigrams): Train Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.82      0.89     12532
           1       0.86      0.98      0.92     14578

    accuracy                           0.90     27110
   macro avg       0.91      0.90      0.90     27110
weighted avg       0.91      0.90      0.90     27110

Evaluating Random Forest (BoW Bigrams) on validation set...

===== Random Forest (BoW Bigrams): Validation Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.81      0.88      2685
           1       0.86      0.97      0.91      3124

    accuracy                           0.90      5809
   macro avg       0.91      0.89      0.90      5809
weighted avg       0.91    

In [24]:
bow_bi_mlp_metrics = fit_pipeline_predict_evaluate(
    model_class=MLPClassifier(
    hidden_layer_sizes=(32,),
    activation='relu',
    solver='adam',
    alpha=0.5,
    learning_rate='adaptive',
    max_iter=100,
    early_stopping=True,
    n_iter_no_change=5,
    random_state=7
),
    model_name="MLP",
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    vectorizer_type='bow',
    ngram_range=(1, 2)
)
print(bow_bi_mlp_metrics)

Fitting MLP...
Predicting on training set...
Predicting on validation set...
Evaluating MLP on training set...

===== MLP: Train Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.98      0.98     12532
           1       0.98      0.99      0.99     14578

    accuracy                           0.99     27110
   macro avg       0.99      0.99      0.99     27110
weighted avg       0.99      0.99      0.99     27110

Evaluating MLP on validation set...

===== MLP: Validation Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.91      0.93      2685
           1       0.92      0.96      0.94      3124

    accuracy                           0.94      5809
   macro avg       0.94      0.93      0.94      5809
weighted avg       0.94      0.94      0.94      5809

  Model  Accuracy  Precision    Recall  F1-score     Dataset
0   MLP  0.986094   0.984974  0

### TF-IDF Vectorizer:

In [25]:
logistic_tfidf_metrics = fit_pipeline_predict_evaluate(
    LogisticRegression(penalty='l2', C=0.1, max_iter=1000),
    "Logistic Regression",
    X_train,
    y_train,
    X_val,
    y_val,
    vectorizer_type='tfidf',
    ngram_range=(1, 1)
)
print(logistic_tfidf_metrics)

Fitting Logistic Regression...
Predicting on training set...
Predicting on validation set...
Evaluating Logistic Regression on training set...

===== Logistic Regression: Train Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.85      0.90     12532
           1       0.88      0.97      0.92     14578

    accuracy                           0.91     27110
   macro avg       0.92      0.91      0.91     27110
weighted avg       0.92      0.91      0.91     27110

Evaluating Logistic Regression on validation set...

===== Logistic Regression: Validation Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.84      0.89      2685
           1       0.88      0.96      0.92      3124

    accuracy                           0.91      5809
   macro avg       0.91      0.90      0.91      5809
weighted avg       0.91      0.91      0.91      5809

            

In [27]:
tfidf_uni_nb_metrics = fit_pipeline_predict_evaluate(
    model_class=MultinomialNB(alpha=7),
    model_name="Multinomial Naive Bayes (TF-IDF Unigrams)",
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    vectorizer_type='tfidf',
    ngram_range=(1, 1)
)

print(tfidf_uni_nb_metrics)

Fitting Multinomial Naive Bayes (TF-IDF Unigrams)...
Predicting on training set...
Predicting on validation set...
Evaluating Multinomial Naive Bayes (TF-IDF Unigrams) on training set...

===== Multinomial Naive Bayes (TF-IDF Unigrams): Train Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.90      0.93     12532
           1       0.92      0.96      0.94     14578

    accuracy                           0.93     27110
   macro avg       0.94      0.93      0.93     27110
weighted avg       0.94      0.93      0.93     27110

Evaluating Multinomial Naive Bayes (TF-IDF Unigrams) on validation set...

===== Multinomial Naive Bayes (TF-IDF Unigrams): Validation Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.89      0.91      2685
           1       0.91      0.95      0.93      3124

    accuracy                           0.92      5809
   macro av

In [28]:
tfidf_uni_rf_metrics = fit_pipeline_predict_evaluate(
    model_class=RandomForestClassifier(
    n_estimators=100,
    max_depth=120,
    min_samples_leaf=5,
    max_features='sqrt',
    random_state=7),
    model_name="Random Forest (TF-IDF Unigrams)",
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    vectorizer_type='tfidf',
    ngram_range=(1, 1)
)

print(tfidf_uni_rf_metrics)

Fitting Random Forest (TF-IDF Unigrams)...
Predicting on training set...
Predicting on validation set...
Evaluating Random Forest (TF-IDF Unigrams) on training set...

===== Random Forest (TF-IDF Unigrams): Train Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.86      0.90     12532
           1       0.89      0.95      0.92     14578

    accuracy                           0.91     27110
   macro avg       0.91      0.91      0.91     27110
weighted avg       0.91      0.91      0.91     27110

Evaluating Random Forest (TF-IDF Unigrams) on validation set...

===== Random Forest (TF-IDF Unigrams): Validation Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.85      0.89      2685
           1       0.88      0.95      0.91      3124

    accuracy                           0.90      5809
   macro avg       0.91      0.90      0.90      5809
weighte

In [29]:
tfidf_uni_mlp_metrics = fit_pipeline_predict_evaluate(
    model_class=MLPClassifier(
    hidden_layer_sizes=(32,),
    activation='relu',
    solver='adam',
    alpha=0.5,
    learning_rate='adaptive',
    max_iter=100,
    early_stopping=True,
    n_iter_no_change=5,
    random_state=7
),
    model_name="MLP",
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    vectorizer_type='bow',
    ngram_range=(1,1)
)
print(tfidf_uni_mlp_metrics)

Fitting MLP...
Predicting on training set...
Predicting on validation set...
Evaluating MLP on training set...

===== MLP: Train Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.94      0.95     12532
           1       0.95      0.98      0.96     14578

    accuracy                           0.96     27110
   macro avg       0.96      0.96      0.96     27110
weighted avg       0.96      0.96      0.96     27110

Evaluating MLP on validation set...

===== MLP: Validation Set =====
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.90      0.92      2685
           1       0.92      0.95      0.94      3124

    accuracy                           0.93      5809
   macro avg       0.93      0.93      0.93      5809
weighted avg       0.93      0.93      0.93      5809

  Model  Accuracy  Precision    Recall  F1-score     Dataset
0   MLP  0.958871   0.949456  0

### Transformer Model

In [70]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from datasets import load_dataset


ModuleNotFoundError: Could not import module 'Trainer'. Are this object's requirements defined correctly?

In [68]:

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(X_val, truncation=True, padding=True, max_length=512)

class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

train_dataset = TextDataset(train_encodings, y_train)
val_dataset = TextDataset(val_encodings, y_val)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    save_strategy="no",
    logging_dir="./logs",
    load_best_model_at_end=False,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)


trainer.train()

preds = trainer.predict(val_dataset)
y_pred = np.argmax(preds.predictions, axis=1)

print(classification_report(y_val, y_pred))


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).